TODO: 
Clean Strings
...


In [1]:
import sys
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import nltk
from nltk import word_tokenize
import time, random, re, pprint
pd.set_option('display.max_columns', None)
print(sys.version_info)

sys.version_info(major=3, minor=5, micro=2, releaselevel='final', serial=0)


## Reading Data and Cleaning

In [2]:
train_input = pd.read_csv('train_input.csv')
train_output = pd.read_csv('train_output.csv')

In [3]:
train_input.head()

,id,conversation
0,0,<speaker_1> seaworld ceo steps down amid tanki...
1,1,<speaker_1> strickland chargers owner dean spa...
2,2,<speaker_1> iniesta plays keepy uppy with one ...
3,3,<speaker_1> chappie trailer #1 <number> hugh j...
4,4,<speaker_1> why the church of satan may get to...


In [4]:
train_input.shape

(165000, 2)

In [8]:
train_output.head()

,id,category
0,0,news
1,1,nfl
2,2,soccer
3,3,movies
4,4,politics


In [90]:
delchars = ''.join(c for c in map(chr, range(256)) if not c.isalpha())
delchars = ''.join(ch for ch in delchars if ch !=' ')
def clean_text(paragraph):
    output = re.sub('\<.*?\>','', paragraph) #delete tags (between <..>)
    output = re.sub('\@.*?\s','', output) #delete usernames (words following @)
    output = re.sub('\n','', output) #delete anything following a slash
    output = output.translate(str.maketrans('','',delchars))
    return output

In [115]:
start = time.time()
text_vector = train_input["conversation"].apply(lambda x: clean_text(x))
train_input["text"] = text_vector
train_input["words"] = train_input["text"].apply(lambda x: x.split()) 
print(time.time() - start)

6.332743167877197


In [116]:
train_input["output"] = train_output["category"]

In [117]:
train_input.head()

,id,conversation,text,words,output
0,0,<speaker_1> seaworld ceo steps down amid tanki...,seaworld ceo steps down amid tanking revenues...,"[seaworld, ceo, steps, down, amid, tanking, re...",news
1,1,<speaker_1> strickland chargers owner dean spa...,strickland chargers owner dean spanos and gol...,"[strickland, chargers, owner, dean, spanos, an...",nfl
2,2,<speaker_1> iniesta plays keepy uppy with one ...,iniesta plays keepy uppy with one leg man yo...,"[iniesta, plays, keepy, uppy, with, one, leg, ...",soccer
3,3,<speaker_1> chappie trailer #1 <number> hugh j...,chappie trailer hugh jackman sci fi comedy...,"[chappie, trailer, hugh, jackman, sci, fi, com...",movies
4,4,<speaker_1> why the church of satan may get to...,why the church of satan may get to open your ...,"[why, the, church, of, satan, may, get, to, op...",politics


In [94]:
train_input["conversation"][1]

'<speaker_1> strickland chargers owner dean spanos and goldman - sachs agree to build la stadium insidestl . com </s> <speaker_2> utkevinacee twitter . com @chasingsunsets7 @utkevinacee any truth to the spanos goldman sachs deal to a build stadium in la ? no . </s> <speaker_3> right . when i look for facts i definitely look to acee s </s> </d>\n'

In [99]:
train_input["text"][1]

' strickland chargers owner dean spanos and goldman  sachs agree to build la stadium insidestl  com   utkevinacee twitter  com any truth to the spanos goldman sachs deal to a build stadium in la  no    right  when i look for facts i definitely look to acee s  '

In [104]:
start = time.time()
train_input["words"] = train_input["text"].apply(lambda x: word_tokenize(x)) 
print(time.time() - start)

67.92844581604004


In [105]:
train_input.head()

,id,conversation,text,words
0,0,<speaker_1> seaworld ceo steps down amid tanki...,seaworld ceo steps down amid tanking revenues...,"[seaworld, ceo, steps, down, amid, tanking, re..."
1,1,<speaker_1> strickland chargers owner dean spa...,strickland chargers owner dean spanos and gol...,"[strickland, chargers, owner, dean, spanos, an..."
2,2,<speaker_1> iniesta plays keepy uppy with one ...,iniesta plays keepy uppy with one leg man yo...,"[iniesta, plays, keepy, uppy, with, one, leg, ..."
3,3,<speaker_1> chappie trailer #1 <number> hugh j...,chappie trailer hugh jackman sci fi comedy...,"[chappie, trailer, hugh, jackman, sci, fi, com..."
4,4,<speaker_1> why the church of satan may get to...,why the church of satan may get to open your ...,"[why, the, church, of, satan, may, get, to, op..."


In [106]:
train_input["words"][1]

['strickland',
 'chargers',
 'owner',
 'dean',
 'spanos',
 'and',
 'goldman',
 'sachs',
 'agree',
 'to',
 'build',
 'la',
 'stadium',
 'insidestl',
 'com',
 'utkevinacee',
 'twitter',
 'com',
 'any',
 'truth',
 'to',
 'the',
 'spanos',
 'goldman',
 'sachs',
 'deal',
 'to',
 'a',
 'build',
 'stadium',
 'in',
 'la',
 'no',
 'right',
 'when',
 'i',
 'look',
 'for',
 'facts',
 'i',
 'definitely',
 'look',
 'to',
 'acee',
 's']